# Asset / Portfolio Returns


## Data preparation

First thing would be to download the required data using [`pandas-datareader`](https://pandas-datareader.readthedocs.io/en/latest) package.  Remember we can easily install it using `pip`:

```
$ pip install pandas_datareader
```

In [1]:
import requests_cache
from datetime import datetime, timedelta
import pandas_datareader.data as web

expire_after = timedelta(days=180)
session = requests_cache.CachedSession(cache_name='cache', backend='sqlite', expire_after=expire_after)
msft = web.DataReader("MSFT", "google", datetime(2015, 12, 25), datetime(2016, 12, 31), session=session)
aapl = web.DataReader("AAPL", "google", datetime(2015, 12, 25), datetime(2016, 12, 31), session=session)
print(msft[:2], "\n")
print(aapl[:2])

             Open   High    Low  Close    Volume
Date                                            
2015-12-28  55.35  55.95  54.98  55.95  22458293
2015-12-29  56.29  56.85  56.06  56.55  27731403 

              Open    High     Low   Close    Volume
Date                                                
2015-12-28  107.59  107.69  106.18  106.82  26704210
2015-12-29  106.96  109.43  106.86  108.74  30931243


As some month-end data might be missing (mainly because holidays), we need to pad the series with latest known values:

In [2]:
month_end_prices = msft.Close.resample("M").pad()
month_end_prices[:5]

Date
2015-12-31    55.48
2016-01-31    55.09
2016-02-29    50.88
2016-03-31    55.23
2016-04-30    49.87
Freq: M, Name: Close, dtype: float64

# Asset Returns

Net Return over month $t$ is defined by $R_t = \frac{P_t - P_{t-1}}{P_{t-1}} = \%\Delta P_{t}$.

Intuitively known as the _percentage change in price_.

In [3]:
net_return = msft.Close['2016-03-31'] / msft.Close['2016-02-29'] - 1
net_return

0.085495283018867774

that is equivalent to use `olhc` function we saw in Pandas introduction:

In [4]:
pt = msft.Close['2016-02-01':'2016-03-31'].resample("M").ohlc().close
pt

Date
2016-02-29    50.88
2016-03-31    55.23
Freq: M, Name: close, dtype: float64

In [5]:
net_return = pt[1] / pt[0] - 1
net_return

0.085495283018867774

Pandas already has a function to calculate it, named `pct_change`. For instance, this calculate the net returns for one month:

In [6]:
one_month_net_returns = month_end_prices.pct_change()
one_month_net_returns[:5]

Date
2015-12-31         NaN
2016-01-31   -0.007030
2016-02-29   -0.076420
2016-03-31    0.085495
2016-04-30   -0.097049
Freq: M, Name: Close, dtype: float64

### Alternative method

Another way to calculate it is using $r_i = {log}(\frac{p_i}{p_{i-1}})$

Note we're using [Series `shift` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.shift.html)

In [7]:
import numpy as np

def daily_returns(closes):    
    return np.log(closes / closes.shift(1))

returns = daily_returns(month_end_prices)
returns[:5]

Date
2015-12-31         NaN
2016-01-31   -0.007054
2016-02-29   -0.079498
2016-03-31    0.082036
2016-04-30   -0.102087
Freq: M, Name: Close, dtype: float64

### K-Month Generalization

Beware of adding two simple one-period returns. Having $R_{t-1} = 0.5$ and $R_t = -0.5$ gives a two-period return of zero, but it should be $R_t(2) - 1 = (1.5 \cdot 0.5) - 1 = -0.25$.

Better to generalize it as:

$1 + R_t(k) = \prod{1 + R_{t-j}}$ 

For instance, the geometric average of two one-month returns, March and April, are:

In [8]:
rt2 = ((returns[3] - 1) * (returns[4] - 1)) - 1
rt2

0.011675480627296064

## Portfolio Returns

The same values we've calculated before for assets can easily be translated to whole _portfolios_.

Let's calculate the initial portfolio value and the amount of shares

In [9]:
msft_shares = 10
aapl_shares = 10
initial_portfolio_value = (msft_shares * msft.Close['2016-03-30']) + (aapl_shares * aapl.Close['2016-03-30'])
initial_portfolio_value

1646.0999999999999

Percentage of each stock, should add to 1

In [10]:
x_msft = (msft_shares * msft.Close['2016-03-30']) / initial_portfolio_value
x_aapl = (aapl_shares * aapl.Close['2016-03-30']) / initial_portfolio_value
[x_msft, x_aapl, x_msft+x_aapl]

[0.3344268270457445, 0.66557317295425544, 1.0]

One-month returns for AAPL and MSFT

In [11]:
ret_msft = msft.Close['2016-04-29'] / msft.Close['2016-03-30'] - 1
ret_aapl = aapl.Close['2016-04-29'] / aapl.Close['2016-03-30'] - 1
[ret_msft, ret_aapl]

[-0.094096276112624833, -0.14439576487769268]

You can see them as _weights_. Using them, the one-month rate of return on the portfolio is

In [12]:
rpt = (x_msft*ret_msft) + (x_aapl*ret_aapl)
rpt

-0.12757426644796796

The portfolio value at the end of month $t$ is $V_t = V_{t-1}(1 + R_{p,t})$

In [13]:
vt = initial_portfolio_value * (1 + rpt)
vt

1436.0999999999999

In general, for a portfolio of $n$ assets with investment shares $x_i$ such that $x_1+...+x_n=1$, the one-period portfolio gross and simple returns are defined as

$R_{p,t} = \sum\limits_{i=1}^n x_i R_{i,t}$